In [3]:
import warnings
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import numpy as np
warnings.filterwarnings("ignore")
np.random.seed(40)

import mlflow
import mlflow.sklearn


In [4]:
import pandas as pd 

# read data from file
df = pd.read_csv("wine-quality.csv")

In [30]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(df)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# make it easy to pass datasets
datasets = { 'train_x': train_x, 'train_y': train_y, 'test_x': test_x, 'test_y': test_y }

In [34]:
def eval_metrics(actual, predicted):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2


In [44]:
def train_elasticnet(in_alpha, in_l1_ratio):
    from sklearn.linear_model import ElasticNet

    def eval_parameters(in_alpha, in_l1_ratio):
        # Set default values if no alpha is provided
        alpha = float(in_alpha) if not in_alpha is None else 0.5
        # Set default values if no l1_ratio is provided
        l1_ratio = float(in_l1_ratio) if not in_l1_ratio is None else 0.5
        return alpha, l1_ratio
 
    alpha, l1_ratio = eval_parameters(in_alpha, in_l1_ratio)
    print("Parameters (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    
    with mlflow.start_run():
        # train with ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)

        # store model
        mlflow.sklearn.log_model(lr, "model")
        
        return rmse

In [45]:
train_elasticnet(0.5, 0.5)

Parameters (alpha=0.500000, l1_ratio=0.500000):
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.8249202019006774
  MAE: 0.6263468839922693
  R2: 0.1294309415517565


0.8249202019006774

In [46]:
train_elasticnet(0.5, 0.4)

Parameters (alpha=0.500000, l1_ratio=0.400000):
Elasticnet model (alpha=0.500000, l1_ratio=0.400000):
  RMSE: 0.8157309265057607
  MAE: 0.6220007658217043
  R2: 0.1487184823372092


0.8157309265057607

In [47]:
train_elasticnet(0.4, 0.4)

Parameters (alpha=0.400000, l1_ratio=0.400000):
Elasticnet model (alpha=0.400000, l1_ratio=0.400000):
  RMSE: 0.8070312918940539
  MAE: 0.6180481283339012
  R2: 0.16677921020746322


0.8070312918940539

In [42]:
!pip install optuna

     |████████████████████████████████| 112kB 45.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 112kB 49.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 12.9MB/s eta 0:00:01
     |████████████████████████████████| 51kB 10.2MB/s eta 0:00:01
  Created wheel for optuna: filename=optuna-0.17.1-cp37-none-any.whl size=144909 sha256=1d6ddb114c1f18a8b99149cfea08e53fbf70e83d0803b1dc7e4dd78e48a9a215
  Stored in directory: /home/ubuntu/.cache/pip/wheels/11/88/8a/f490ae6d6bea4398e52b38d122f46fa9287d14c0a3cf6ba728
  Created wheel for PrettyTable: filename=prettytable-0.7.2-cp37-none-any.whl size=13700 sha256=0adc8564597c8db7df1510569b732d09106ce23fe023a66a6011e44569431757
  Stored in directory: /home/ubuntu/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Created wheel for pyperclip: filename=pyperclip-1.7.0-cp37-none-any.whl size=8359 sha256=4d7f1cc501b92c4a54774b8669d82712244

In [52]:
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)
# Define an objective function to be minimized.
def objective(trial):

    suggested_alpha = trial.suggest_uniform('alpha', 0.1, 0.8)
    suggested_l1_ratio = trial.suggest_uniform('l1_ratio', 0.1, 0.8)
        
    error = train_elasticnet(suggested_alpha, suggested_l1_ratio)
    
    return error  # A objective value linked with the Trial object.

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100)  # Invoke optimization of the objective function.

Parameters (alpha=0.254045, l1_ratio=0.473723):
Elasticnet model (alpha=0.254045, l1_ratio=0.473723):
  RMSE: 0.7968764774625045
  MAE: 0.6137436021901664
  R2: 0.18761599620559655


[I 2019-10-27 22:57:11,409] Finished trial#0 resulted in value: 0.7968764774625045. Current best value is 0.7968764774625045 with parameters: {'alpha': 0.2540445179937741, 'l1_ratio': 0.4737226705341375}.


Parameters (alpha=0.289524, l1_ratio=0.527144):
Elasticnet model (alpha=0.289524, l1_ratio=0.527144):
  RMSE: 0.8028698118269084
  MAE: 0.6162333376728204
  R2: 0.17535010897514758


[I 2019-10-27 22:57:11,609] Finished trial#1 resulted in value: 0.8028698118269084. Current best value is 0.7968764774625045 with parameters: {'alpha': 0.2540445179937741, 'l1_ratio': 0.4737226705341375}.


Parameters (alpha=0.471290, l1_ratio=0.488196):
Elasticnet model (alpha=0.471290, l1_ratio=0.488196):
  RMSE: 0.8203965494890564
  MAE: 0.6241973999577919
  R2: 0.13895272050831542


...

[I 2019-10-27 22:57:31,246] Finished trial#97 resulted in value: 0.7871325748272701. Current best value is 0.7851926259536915 with parameters: {'alpha': 0.1001720071933421, 'l1_ratio': 0.1005012293821135}.


Parameters (alpha=0.355528, l1_ratio=0.128279):
Elasticnet model (alpha=0.355528, l1_ratio=0.128279):
  RMSE: 0.7920412715871928
  MAE: 0.6128568157248606
  R2: 0.19744468842243457


[I 2019-10-27 22:57:31,452] Finished trial#98 resulted in value: 0.7920412715871928. Current best value is 0.7851926259536915 with parameters: {'alpha': 0.1001720071933421, 'l1_ratio': 0.1005012293821135}.


Parameters (alpha=0.100450, l1_ratio=0.235066):
Elasticnet model (alpha=0.100450, l1_ratio=0.235066):
  RMSE: 0.7865967154589315
  MAE: 0.6120108226622007
  R2: 0.20844042626966508


[I 2019-10-27 22:57:31,670] Finished trial#99 resulted in value: 0.7865967154589315. Current best value is 0.7851926259536915 with parameters: {'alpha': 0.1001720071933421, 'l1_ratio': 0.1005012293821135}.


In [54]:
study.best_params

{'alpha': 0.1001720071933421, 'l1_ratio': 0.1005012293821135}